# 07-Edge Matching

This notebooks shows how to match road network edges to the EVED map-matched GPS locations using three different methods:
1. OSMnx API
2. Triangle inequality query with fit metric
3. Triangle inequality query with distance metric

**Requirements**: Run the `calculate-trajectories.py` script before running this notebook.

In [1]:
import numpy as np
import osmnx as ox
import networkx as nx
import utm

from itertools import pairwise
from geo.math import num_haversine, vec_haversine
from geo.road import RoadNetwork, download_road_network, download_road_network_bbox
from geo.trajectory import load_trajectory_points

In [2]:
road_network = download_road_network_bbox(42.325853, 42.220268, -83.804839, -83.673437)

# road_network = download_road_network("Ann Arbor, Michigan, USA")

## Matching Edges with OSMnx

We start by using OSMnx's own functions to find edge matches to locations sampled from the EVED. The first thing we need to do is to project the whole road network to the local UTM projection.

In [3]:
network_utm = ox.projection.project_graph(road_network)

The above operation converts latitudes and longitudes to a local UTM projection that works as a Cartesian plane in meters. The advantage of such projection is that you can directly use your knowledge of vectors to work out distances and other planar geometry calculations. The disadvantage is that you always need to perform the conversion before using OSMnx's functions. When converting from (_latitude_, _longitude_) to UTM, we get a converted coordinate pair (_x_, _y_) aptly named (_easting_, _northing_), along with the UTM number and letter codes. We might need these later to convert (_easting_, _northing_) back to (_latitude_, _longitude_).

In [4]:
easting, northing, zone_num, zone_ltr = utm.from_latlon(42.287702, -83.707775)

We can now call OSMnx's `nearest_edges` [function](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_edges) to determine the closest edges.

In [5]:
edge_id = ox.distance.nearest_edges(network_utm, easting, northing)

In [6]:
edge_id

(62505058, 4306881815, 0)

The first two numbers in the tuple identify the edge's nodes, and we can query it from the network using the following:

In [7]:
network_utm[edge_id[0]][edge_id[1]][0]

{'osmid': 8723817,
 'oneway': False,
 'lanes': '2',
 'highway': 'tertiary',
 'reversed': True,
 'length': 116.428,
 'speed_kph': 48.3,
 'travel_time': 8.7,
 'bearing': 267.3,
 'name': 'Glazier Way',
 'maxspeed': '30 mph'}

In [8]:
network_utm[edge_id[1]][edge_id[0]][0]

{'osmid': 8723817,
 'oneway': False,
 'lanes': '2',
 'highway': 'tertiary',
 'reversed': False,
 'length': 116.428,
 'speed_kph': 48.3,
 'travel_time': 8.7,
 'bearing': 87.3,
 'name': 'Glazier Way',
 'maxspeed': '30 mph'}

In [9]:
trajectory = load_trajectory_points(4, unique=True)
raw_lists = map(list, zip(*trajectory))
latitudes, longitudes, bearings = map(np.array, raw_lists)

In [10]:
eastings, northings, zone_num, zone_ltr = utm.from_latlon(latitudes, longitudes)

In [11]:
len(eastings), len(northings)

(203, 203)

## Performance Metrics

**OSMnx 1.3.0**

In [12]:
%%timeit
ox.distance.nearest_edges(network_utm, eastings, northings)

3.3 s ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Fitness Ratio**

In [13]:
%%timeit
rn = RoadNetwork(road_network)
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_matching_edge(*p)
        cache[p] = edge

465 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Distance**

In [14]:
%%timeit
rn = RoadNetwork(road_network)
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_nearest_edge(*p)
        cache[p] = edge

487 ms ± 8.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
edges_0 = ox.distance.nearest_edges(network_utm, eastings, northings)

In [16]:
rn = RoadNetwork(road_network)
edges_1 = []
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_matching_edge(*p)
        cache[p] = edge
    edges_1.append(cache[p])

In [17]:
rn = RoadNetwork(road_network)
edges_2 = []
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_nearest_edge(*p)
        cache[p] = edge
    edges_2.append(cache[p])

In [18]:
e0 = [(e[0], e[1]) for e in edges_0 if e is not None]
e1 = [(e[0], e[1]) for e in edges_1 if e is not None]
e2 = [(e[0], e[1]) for e in edges_2 if e is not None]

In [19]:
len(e0), len(e1), len(e2)

(203, 203, 203)

In [20]:
max([road_network[e[0]][e[1]][0]["length"] for e in road_network.edges if "length" in road_network[e[0]][e[1]][0]])

608.873

In [21]:
ds = []
for e in road_network.edges:
    n0 = road_network.nodes[e[0]]
    n1 = road_network.nodes[e[1]]
    ds.append(num_haversine(n0['y'], n0['x'], n1['y'], n1['x']))

In [22]:
max(ds)

609.5543654244057

In [23]:
road_network[62503307]

AdjacencyView({62476299: {0: {'osmid': 8859758, 'lanes': '4', 'name': 'Ann Arbor-Saline Road', 'highway': 'secondary', 'maxspeed': '45 mph', 'oneway': False, 'reversed': False, 'length': 49.169, 'speed_kph': 72.4, 'travel_time': 2.4, 'bearing': 221.1}}, 64349821: {0: {'osmid': 8859758, 'lanes': '4', 'name': 'Ann Arbor-Saline Road', 'highway': 'secondary', 'maxspeed': '45 mph', 'oneway': False, 'reversed': True, 'length': 127.553, 'speed_kph': 72.4, 'travel_time': 6.3, 'bearing': 42.4}}})

In [24]:
road_network.nodes[26950035]

{'y': 42.3001518, 'x': -83.6871173, 'street_count': 2}

In [25]:
ys = set()
xs = set()
for node in road_network.nodes:
    ys.add(road_network.nodes[node]['y'])
    xs.add(road_network.nodes[node]['x'])

In [26]:
min(ys), max(ys), min(xs), max(xs)

(42.2202745, 42.3258498, -83.8048349, -83.6734421)

In [27]:
list(road_network.adj[62503312])

[6093040500, 4659829181, 4659847762, 4659847759]

In [28]:
nx.shortest_path(road_network, 62503312, 9871253637)

[62503312, 4659847762, 9871253637]

In [29]:
nx.shortest_path(road_network, 62476295, 62476294)

[62476295, 62476294]

In [30]:
nx.number_connected_components(nx.to_undirected(road_network))

19

In [31]:
nx.has_path(road_network, 62504320, 62475322)

False